## Import libraries

In [31]:
import os
import pandas as pd
import torch

from transformers import BertTokenizer

## Import dataset

In [32]:
project_dir = os.path.dirname(os.getcwd())

In [33]:
df = pd.read_csv(os.path.join(project_dir, 'data', 'processed', 'processed_data.csv'))

In [34]:
df.head()

,Unnamed: 0.1,Unnamed: 0,statement,status,num_of_characters,num_of_sentences,cleaned_statement
0,0,0,oh my gosh,Anxiety,10,1,oh gosh
1,1,1,"trouble sleeping, confused mind, restless hear...",Anxiety,64,2,trouble sleeping confused mind restless heart ...
2,2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,78,2,wrong back dear forward doubt stay restless re...
3,3,3,I've shifted my focus to something else but I'...,Anxiety,61,1,ive shifted focus something else im still worried
4,4,4,"I'm restless and restless, it's been a month n...",Anxiety,72,2,im restless restless month boy mean


In [35]:
df.isnull().sum()

Unnamed: 0.1           0
Unnamed: 0             0
statement              0
status                 0
num_of_characters      0
num_of_sentences       0
cleaned_statement    131
dtype: int64

Looks like there were some statements which were empty after cleaning. After reading the file, they are read as NaNs. Remove these rows.

## Text Vectorization
Turn text into numerical representations

In [26]:
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')